## Anomaly detection and pattern extraction with Spark, Cassandra and Scala

Today, geo-located data is available in a number of domains, ranging from healthcare to financial markets, to social services. In all these domains, extracting patterns and detecting anomalies and novelties from data has very concrete business outcomes. 

Anomaly detection can be defined as the process of finding which samples in the given dataset do not follow the given patterns and behave as though they were produced by a different mechanism. From detection follows action. Depending on the domain and the use case, we define them as anomalies or novelties and these signals are the triggers for applications such as personalized marketing and fraud alerting and notification.

As more data gets ingested/produced via digital services, it’s key to perform this sort of analytics at scale. In the open source space, technologies such as Spark and Cassandra are definitely instrumental to implement and execute modern data pipelines at scale.

#### Synopsis

In this Oriole, we will collect data from Cassandra and bring it up to Spark for further analysis. We will see how to prepare and analyze the data using a combination of Spark and Cassandra exploratory queries. Then we will perform the following data analyses:

  - Descriptive statistics 
  - Events histograms
  - Extract users' preferences
  - Detect novel/anomalous events

#### References and datasets

For this analysis, we are going to use the Gowalla Dataset [1]. The Gowalla dataset consists of a table of events, registered by anonymized users. Each event registers a user checking into a geolocated venue at a specific timestamp. The dataset is available at https://snap.stanford.edu/data/loc-gowalla.html

A number of venues in this demo have been tagged with an actual name. Thanks to the https://code.google.com/archive/p/locrec/ project (now archived). The project is being developed in the context of the SInteliGIS project financed by the Portuguese Foundation for Science and Technology (FCT) through project grant PTDC/EIA-EIA/109840/2009.

[1] E. Cho, S. A. Myers, J. Leskovec. Friendship and Mobility: Friendship and Mobility: User Movement in Location-Based Social Networks ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 2011.



#### Setup

This notebook is running scala code and interfaces to a Spark cluster using the [Apache Toree](https://toree.incubator.apache.org/) project. Furthermore, Spark reads the data from Cassandra tables. Spark interfaces to Cassandra via the [Cassandra-Spark connector](https://github.com/datastax/spark-cassandra-connector). 

At the time of compiling this notebook, Spark 1.6.1 and Cassandra 3.5 were used. Here below the command to install the Spark - Scala Kernel on Jupiter. More instructions on this topic are available on Apache Toree [website](https://toree.incubator.apache.org/) and [github pages](https://github.com/apache/incubator-toree).

```
sudo jupyter-toree install --spark_home=${SPARK_HOME} \
--spark_opts='--packages com.datastax.spark:spark-cassandra-connector_2.10:1.6.0 --conf spark.cassandra.connection.host=localhost'
```

_Please allow a few seconds for the next cell to complete.  
It will start a single-node Spark context on the Oriole container, and connect it to the Oriole scala code cells._

In [1]:
// Scala version
sc.version

2.1.0

##### Quick Scala Spark Tests

In [2]:
val NUM_SAMPLES = 1000000
val count = sc.parallelize(1 to NUM_SAMPLES).filter { _ =>
  val x = math.random
  val y = math.random
  x*x + y*y < 1
}.count()
println(s"Pi is roughly ${4.0 * count / NUM_SAMPLES}")

Pi is roughly 3.143784


#### spark session

In [3]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()

// For implicit conversions like converting RDDs to DataFrames
import spark.implicits._

### Connecting to Cassandra

In [4]:
// spark-cassandra connector
import com.datastax.spark.connector._
import com.datastax.spark.connector.cql._

#### SQL queries with Spark and Cassandra

Cassandra is exposed via a SQL context, so there is not need to learn a separate syntax as Spark will map the query to the available features of the underlying storage system. See below a simple query accessing the name and the id of venues from a cassandra table. Also remember that sql statements are _staged_ but not _executed_ until some actual [actions](http://spark.apache.org/docs/latest/programming-guide.html#actions) needs to be computed. Examples of actions are for instance, **count**(), **first**(), **collect**().

#### Dataset: Venues

Let's first have a quick look at the venues. They are stored in the cassandra table `lbsn.venues`. 

In [5]:
val venues = spark.
               read.
               format("org.apache.spark.sql.cassandra").
               options(Map( "table" -> "venues", "keyspace" -> "lbsn" )).
               load()

Let's start by counting the number of venues, and selecting vid `12525`, to get a feeling about how Spark works and getting some facts about the dataset.
Feel free to modify the below cells to gain insight on the venue data. Try for instance the `take()` and the `show()` spark dataframe methods.

In [6]:
venues.count()

17291

In [7]:
venues.where("vid = 12525").first()

[12525,40.7612551699,-73.977579698,The Museum of Modern Art (MoMA)]

#### Executing queries

Spark and Cassadra work together when it comes to executing queries. If the query can be executed directly on the database, Spark will offload the query to Cassandra. However, not all queries can be fully performed on cassandra, and that's where the combination Spark-Cassandra gets really handy. For instance, when executing joins, Spark will partition and plan the query _pushing down_ what can be done in Cassandra and perform in Spark the rest of the query. 

More information can be found on Cassandra Documentation about [using Spark SQL to query data](http://docs.datastax.com/en/datastax_enterprise/5.0/datastax_enterprise/spark/sparkSqlOverview.html) or on the [Cassandra Spark Connector](https://github.com/datastax/spark-cassandra-connector) pages.

#### Joining Cassandra tables with Spark

The query here below filters out those events which were registered in the New York City area. As filtering in cassandra cannot by done directly (lat/lon columns which are not indexed in this example), this specific query will first move the data form Cassandra to Spark, and then will perform the filtering in Spark. 

In general, it's a good practice to push down and filter as much data as early as possible. This practice keeps the throughput low and minimize the data transfered from one system to the other.

#### Dataset: Events

Let's first have a quick look at the events. They are stored in the cassandra table `lbsn.events`. 

In [8]:
spark.sql("DROP VIEW IF EXISTS events")

val createDDL = """
     CREATE TEMPORARY VIEW events
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "events",
     keyspace "lbsn",
     pushdown "true")"""

// Creates Catalog Entry registering an existing Cassandra Table
spark.sql(createDDL);

[]

In [9]:
val events   = spark.sql("""select ts, uid, lat, lon, vid from events where
                            lon>-74.2589 and lon<-73.7004 and 
                            lat> 40.4774 and lat< 40.9176
                      """).as("events").orderBy("uid", "ts").repartition(8, $"uid").cache()

In [10]:
events.printSchema()

root
 |-- ts: timestamp (nullable = true)
 |-- uid: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- vid: long (nullable = true)



As you can see here above, Spark dataframe can extract the schema from the Cassandra/Spark SQL query. In this particular case, they mapping of cassandra types to Spark Dataframe types is performed by the [Cassandra-Spark connector](https://github.com/datastax/spark-cassandra-connector). Each event consists in a timestamp, the user and venue id and the location of the event (latitude and longitude).


#### Explore the events data

Let's start by looking at some recorded events. A quick way to do so is with the `show()` Dataframe spark method.
Feel free to modify the below cells to gain insight on the venue data. Try for instance the `take()`, `show()`, `count()` spark dataframe methods.

In [11]:
events.show(5)

+--------------------+---+-------------+------------------+------+
|                  ts|uid|          lat|               lon|   vid|
+--------------------+---+-------------+------------------+------+
|2010-04-18 21:23:...| 88| 40.645919692|    -73.7779355049| 17417|
|2010-04-18 22:33:...| 88|40.7628367545|    -73.9825987816|150676|
|2010-04-18 23:38:...| 88|    40.763479|-73.97908000000002|972311|
|2010-04-19 12:04:...| 88|40.7417466987|     -73.993421425|105068|
|2010-04-19 13:53:...| 88|40.7422661063|-73.98375749590001|851661|
+--------------------+---+-------------+------------------+------+
only showing top 5 rows



Before diving into anomaly detection of geo-located data, let's perform some more basic queries.  
Herebelow, it is shown how to count events registered by user `uid=0`.

In [12]:
// User 0: how many check-ins?

events.where("uid=0").count()

25

#### Joining Cassandra tables in Spark.

One of the advantages of connecting Cassandra and Spark, is the fact that you can now merge and join Cassandra tables. You can actually keep coding with Spark DataFrames, as the join will happen in the background and the necessary queries will be pushed back from Spark to Cassandra.

In [13]:
val df_ny = events.
  join(venues, events("vid") === venues("vid"), "inner").
  select("ts", "uid", "events.lat", "events.lon", "events.vid","name")

Our spark DataFrame df_ny above is going to be the starting point for our data analysis.     Each row records the event's timestamp, the user id, the geo-location (latitude and longitude) of the event and finally the venue's id and venue's name.  

Here below let's have a look at 5 events checkd in by `uid=0`, in no particular order.

In [14]:
df_ny.filter($"uid"===0).show(5,false)

+---------------------+---+-------------+------------------+-----+---------------------------------+
|ts                   |uid|lat          |lon               |vid  |name                             |
+---------------------+---+-------------+------------------+-----+---------------------------------+
|2010-10-07 15:27:40.0|0  |40.6438845363|-73.78280639649999|23261|JFK John F. Kennedy International|
|2010-10-07 20:14:44.0|0  |40.7515076167|-73.9755          |34484|Chrysler Building                |
|2010-10-07 20:31:48.0|0  |40.7484436586|-73.9857316017    |12313|Empire State Building            |
|2010-10-07 21:02:01.0|0  |40.7458101407|-73.98822069170002|60450|Ace Hotel                        |
|2010-10-07 21:58:31.0|0  |40.7422010764|-73.9879953861    |17710|Madison Square Park              |
+---------------------+---+-------------+------------------+-----+---------------------------------+
only showing top 5 rows



#### Executing SQL with custom defined functions.

Spark dataframes can also be filtered and transformed programmatically via a number of [pre-defined functions](https://spark.apache.org/docs/1.6.1/api/scala/#org.apache.spark.sql.functions$), such as min, sum, stddev, and many more. Some of those are shown in the next code sections. 

Next to the default set of pre-defined dataframe and column functions, it is possible to define the user-defined-functions (udf's). In the code below, we will create two UDF's to transform the timestamp to the day of the week and the hour of the day values, computed according to a given local timezone.

In [15]:
// UDF functions for SQL-like operations on columns
import org.joda.time.DateTime
import org.joda.time.DateTimeZone

import java.sql.Timestamp
import org.apache.spark.sql.functions.udf

val  dayofweek = udf( (ts: Timestamp, tz: String) => {
  val dt = new DateTime(ts,DateTimeZone.forID(tz))
  // Monday starts at 1, but we would like to count from zero
  dt.getDayOfWeek() -1
})

val  localhour = udf( (ts: Timestamp, tz: String) => {
  val dt = new DateTime(ts,DateTimeZone.forID(tz))
  dt.getHourOfDay()
})

As you can see below, our newly defined functions `dayofweek` and `localhour` can be mixed and matched with other Spark SQL functions. 

In [16]:
import org.apache.spark.sql.functions.lit
val newyork_tz = "America/New_York"

val df = df_ny.
  withColumn("dow",  dayofweek($"ts", lit(newyork_tz))).
  withColumn("hour", localhour($"ts", lit(newyork_tz))).
  as("events")

df.printSchema()

root
 |-- ts: timestamp (nullable = true)
 |-- uid: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- vid: long (nullable = true)
 |-- name: string (nullable = true)
 |-- dow: integer (nullable = true)
 |-- hour: integer (nullable = true)



In [17]:
// select time and the extracted day of the week and day of the month
df.select("ts", "dow", "hour").show(5, false)

+---------------------+---+----+
|ts                   |dow|hour|
+---------------------+---+----+
|2010-04-18 21:23:48.0|6  |17  |
|2010-04-18 22:33:48.0|6  |18  |
|2010-04-18 23:38:56.0|6  |19  |
|2010-04-19 12:04:48.0|0  |8   |
|2010-04-19 13:53:21.0|0  |9   |
+---------------------+---+----+
only showing top 5 rows



## Extracting user's preferences

#### Basic statistics in Spark

The following code section shows how to collect global statistics and histograms per hour of the day and per day of the week. Histograms can be made more specific by aggregating the events according to a number of factors, such as:

 - venue
 - geographical area
 - popular users
 - 1st, 2nd friend's circle
 
If you are interested, in multiple slicing and dicing option, Spark as a [cube function](https://spark.apache.org/docs/1.5.1/api/scala/index.html#org.apache.spark.sql.DataFrame) as well.  
To start, let's compute the histogram, accumulating all events and aggregating by hour of the day and by day of the week.  


In [18]:
// histogram day of the week events
df.groupBy($"dow").count().show()

+---+-----+
|dow|count|
+---+-----+
|  1|15385|
|  6|15849|
|  3|16179|
|  5|18085|
|  4|16557|
|  2|15687|
|  0|14640|
+---+-----+



In [19]:
// histogram hour of the day events
df.groupBy($"hour").count().show(24,false)

+----+-----+
|hour|count|
+----+-----+
|12  |7257 |
|22  |4747 |
|1   |1272 |
|13  |8142 |
|16  |7519 |
|6   |937  |
|3   |554  |
|20  |7167 |
|5   |550  |
|19  |8970 |
|15  |7254 |
|17  |7495 |
|9   |4586 |
|4   |323  |
|8   |3615 |
|23  |3390 |
|7   |2157 |
|10  |4903 |
|21  |6127 |
|11  |5575 |
|14  |7688 |
|2   |852  |
|0   |2681 |
|18  |8621 |
+----+-----+



#### Statistics in Spark: venue-specific histograms

Here above we have extracted general patterns and looked at the histograms of events across all users and all venues. Moving on, let's have a look on how to create an specific histogram for each venue. 

#### Working with Vectors as DataFrame elements
Working with vectors as DataFrame elements is a very powerful modeling technique, which by the way is extensively used in Spark ML. Having vectors as elements, allows you to use many vector arithmetic and logical operations as well as a great list of vector transformation as provided in the Spark ML library: http://spark.apache.org/docs/latest/ml-features.html, while you still keep the data in a DataFrame. 

In this tutorial, we will store the day and month histogram as a vector. First, let's convert the day-of-the-week to a vector.

In [20]:
import breeze.linalg._
import breeze.linalg.DenseVector

import org.apache.spark.mllib.linalg.{Vector,Vectors}

def r(x: Double, d:Int) = { 
    import scala.math.{pow, round}
    val p = pow(10,d); round(x*p)/p 
}

In [21]:
// vector histogram

def toVector(i: Int, length:Int) = {
  DenseVector((0 to length-1).map(x => if (x == i) 1.0 else 0.0).toArray)
}

Vectors are monday to sundays, ( values go from 0 to 6)  
As an example let's see which vector is produced from this event on Friday.  
the value 4, turns in to a '1' on the 5th position in the day of the week vector: it is a Friday :)

In [22]:
df.select("vid", "ts", "dow").first()

[17417,2010-04-18 21:23:48.0,6]

In [23]:
df.select($"vid", $"dow").rdd.map(r => (r.getLong(0),toVector(r.getInt(1), 7))).first()

(17417,DenseVector(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0))

#### Pair RDDs: reduce by Key

We will now `reduceByKey` those weekly and daily vectors by applying vectors arithmetics. In this way, we can collect the probability of an event happening at a specific day of the week for each venue in the dataset. This is a much more detailed analysis since different venues, such as restaurants, musea and train stations have different daily and weekly histogram patterns.

In [24]:
val dow_hist = df.
  select($"vid", $"dow").
  rdd.
  map(r => (r.getLong(0),toVector(r.getInt(1), 7))).
  reduceByKey(_ + _).
  mapValues(x => Vectors.dense((x / sum(x)).toArray.map(r(_,2)))).
  toDF("vid", "dow_hist")

Here below, the aggregated histogram, for a number of venues, where histogram values have been normalized so that their sum is one (such as in a descrete propability mass function https://en.wikipedia.org/wiki/Probability_mass_function)

In [25]:
dow_hist.show(3, false)

+------+-------------------------------+
|vid   |dow_hist                       |
+------+-------------------------------+
|513544|[0.67,0.0,0.0,0.0,0.0,0.33,0.0]|
|178464|[0.0,0.0,0.0,0.0,1.0,0.0,0.0]  |
|932648|[0.25,0.0,0.0,0.5,0.0,0.0,0.25]|
+------+-------------------------------+
only showing top 3 rows



#### Scoring events according to venues histograms

Now we can use the day of the week histogram dataframe, to correlate when during the day a certain event occur. First let's join this dataframe with the dataframe of the events. 

In [26]:
import org.apache.spark.sql.functions.round

val df_probs = df.
  join(dow_hist, df("vid") === dow_hist("vid"), "inner").
  select($"ts", $"uid", round($"lat",4), round($"lon",4), $"events.vid", $"dow", $"dow_hist")

df_probs.show(5,false)

+---------------------+-----+-------------+-------------+-----+---+---------------------------------+
|ts                   |uid  |round(lat, 4)|round(lon, 4)|vid  |dow|dow_hist                         |
+---------------------+-----+-------------+-------------+-----+---+---------------------------------+
|2010-07-17 23:36:46.0|4750 |40.712       |-74.0096     |11745|5  |[0.1,0.06,0.1,0.06,0.16,0.42,0.1]|
|2010-01-30 16:08:43.0|36070|40.712       |-74.0096     |11745|5  |[0.1,0.06,0.1,0.06,0.16,0.42,0.1]|
|2010-03-13 14:25:18.0|49911|40.712       |-74.0096     |11745|5  |[0.1,0.06,0.1,0.06,0.16,0.42,0.1]|
|2010-07-08 19:38:12.0|71895|40.712       |-74.0096     |11745|3  |[0.1,0.06,0.1,0.06,0.16,0.42,0.1]|
|2010-02-13 22:51:17.0|22   |40.712       |-74.0096     |11745|5  |[0.1,0.06,0.1,0.06,0.16,0.42,0.1]|
+---------------------+-----+-------------+-------------+-----+---+---------------------------------+
only showing top 5 rows



#### Profiling of users according to the visiting trend of venues

Now we have sufficient ingredients to profile user behaviour, relative to the venue statistics. For instance, we can determine if a certain user prefer to go to a venue during peak hours of if he/she prefers to go there when it's quite. Everyone of us has certain rythms, habits and preferentces during the day and this analysis correlates the personal preferences with the venue histogram event distribution.

As an example, we are going to calculate if a given user prefer to visit a given venue during peak hours, or rather off-peak when it's quiter. To do so we are going to calculate how "trendy" is a place during a given day of the week and add this "feature" to our events table. Once we have done it, we can start scoring user preferences.

In [27]:
val  nth = udf( (i:Int, arr: Vector) => {
  val v = arr.toArray.lift(i).getOrElse(0.0) 
  // more or less than average?
  v * arr.toArray.length
})

df_probs.select($"ts", $"uid", $"vid", $"dow", round(nth($"dow", $"dow_hist"),2).as("dow_trend")).show(3,false)

+---------------------+-----+-----+---+---------+
|ts                   |uid  |vid  |dow|dow_trend|
+---------------------+-----+-----+---+---------+
|2010-07-17 23:36:46.0|4750 |11745|5  |2.94     |
|2010-01-30 16:08:43.0|36070|11745|5  |2.94     |
|2010-03-13 14:25:18.0|49911|11745|5  |2.94     |
+---------------------+-----+-----+---+---------+
only showing top 3 rows



#### Trendy places during a day of the week and hour of the day: putting it all together

Let's repeat the same exercise for the histograms binned by hour of the day. And finally, let's merge and compute the probability of each given event, given the venue, the hour of the day, and the day of the week. 

In [28]:
// same for hour of the day

val hour_hist = df.
  select($"vid", $"hour").
  rdd.
  map(r => (r.getLong(0),toVector(r.getInt(1), 24))).
  reduceByKey(_ + _).
  mapValues(x => Vectors.dense((x / sum(x)).toArray.map(r(_,2)))).
  toDF("vid", "hour_hist")

hour_hist.show(1, false)

+------+----------------------------------------------------------------------------------------------------+
|vid   |hour_hist                                                                                           |
+------+----------------------------------------------------------------------------------------------------+
|513544|[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,0.33,0.33,0.0,0.0,0.0]|
+------+----------------------------------------------------------------------------------------------------+
only showing top 1 row



In [29]:
val df_probs = df.
  join(dow_hist, df("vid") === dow_hist("vid"), "inner").
  join(hour_hist, df("vid") === hour_hist("vid"), "inner").
  select( 
    $"ts", 
    $"uid", 
    round($"lat",4), 
    round($"lon",4),
    $"events.vid",
    round(nth($"hour", $"hour_hist"),2).as("hour_trend"), 
    round(nth($"dow",  $"dow_hist"),2).as("dow_trend"))

df_probs.show(5,false)

+---------------------+-----+-------------+-------------+-----+----------+---------+
|ts                   |uid  |round(lat, 4)|round(lon, 4)|vid  |hour_trend|dow_trend|
+---------------------+-----+-------------+-------------+-----+----------+---------+
|2010-07-17 23:36:46.0|4750 |40.712       |-74.0096     |11745|2.4       |2.94     |
|2010-01-30 16:08:43.0|36070|40.712       |-74.0096     |11745|3.12      |2.94     |
|2010-03-13 14:25:18.0|49911|40.712       |-74.0096     |11745|1.44      |2.94     |
|2010-07-08 19:38:12.0|71895|40.712       |-74.0096     |11745|5.52      |0.42     |
|2010-02-13 22:51:17.0|22   |40.712       |-74.0096     |11745|1.44      |2.94     |
+---------------------+-----+-------------+-------------+-----+----------+---------+
only showing top 5 rows



#### Example:

Let's have a look at the most visited places for user 22.  
This can be easily down by filtering the `df_probs` for uid=22 and then group by venue and picking the top 5 venues. In code here below:

In [30]:
val uid = 22

val df_userpref = df_probs.filter($"uid" === uid)
val df_user_topvenues = df_userpref.groupBy("vid").count().sort($"count".desc)
df_user_topvenues.show(5)

+-----+-----+
|  vid|count|
+-----+-----+
|12821|   40|
|12579|   13|
|23261|   13|
|12505|   11|
|12506|    8|
+-----+-----+
only showing top 5 rows



In [31]:
df_userpref.show(5)

+--------------------+---+-------------+-------------+------+----------+---------+
|                  ts|uid|round(lat, 4)|round(lon, 4)|   vid|hour_trend|dow_trend|
+--------------------+---+-------------+-------------+------+----------+---------+
|2010-02-13 22:51:...| 22|       40.712|     -74.0096| 11745|      1.44|     2.94|
|2009-11-11 22:16:...| 22|      40.7292|     -73.9814| 55671|      2.64|     0.77|
|2010-02-18 15:03:...| 22|      40.7475|     -73.9787| 62327|      3.12|     1.75|
|2009-11-17 20:51:...| 22|      40.7224|     -73.9887| 83615|      7.92|     2.31|
|2010-02-13 23:04:...| 22|      40.7161|     -74.0042|101552|       6.0|     1.75|
+--------------------+---+-------------+-------------+------+----------+---------+
only showing top 5 rows



In [32]:
val topvenue = df_user_topvenues.first()

In [33]:
val top_vid    = topvenue(0)
top_vid

12821

In [34]:
val top_vcount = topvenue(1)
top_vcount

40

In [35]:
val top_vname  = venues.where($"vid" === top_vid).first()(1)
top_vname

40.7657052487

In [36]:
println(s"User $uid visited vid: $top_vid ($top_vname), ${top_vcount} times")

User 22 visited vid: 12821 (40.7657052487), 40 times


Now lets analysing visiting patterns for user 22 on the top venue:

In [37]:
val topvenue = df_user_topvenues.first()

val top_vid    = topvenue(0)
val top_vcount = topvenue(1)
val top_vname  = venues.where($"vid" === top_vid).first()(1)

println(s"User $uid visited vid: $top_vid ($top_vname), ${top_vcount} times")

User 22 visited vid: 12821 (40.7657052487), 40 times


#### Collecting user preferences about this user in this place

In [ ]:
df_userpref.filter($"vid" === top_vid).select("dow_trend", "hour_trend").describe().show()

#### Remarks / Considerations

From the statistics above seems that user 22, usually goes to his/her most favorite venue (Manhattan Park) when it's usually more crowded than usual. Definitely a social outgoing person :) . It seems therefore that this user seldom goes to the park when it's less crowded. 

Where to go from here? You could explore more features and statistics and start building a predictive model for each user on when and where a person might register an event. Looking at personal, venues, and other factors you could also establish if a given event is special or anomalous provided user preferences and habits and other venue statistics.



From here, you could also go further in the analysis, taking into consideration the relationships between users (for instance the friend user graph) or the relations between venues, either by looking at the sequence of the events of by looking at their geo-spacial proximity, by creating a density clustering on the venues. Some of these ideas are explained in a companion notebook "Geo-located Clustering and Sequence Mining with Spark, Cassandra and Scala"
